In [4]:
from FundData import *
from helper import Helper
import pprint, json
import fitz

#dir_path = r"C:\Users\Kaustubh.keny\OneDrive - Cogencis Information Services Ltd\Documents\mywork-repo"
dir_path = r"C:\Users\rando\OneDrive\Documents\mywork-repo"

dry_path = r'\output\DryRun.pdf'
fin_path = r'\files\financial_indices.xlsx'
#fund_path = r"C:\Users\Kaustubh.keny\OneDrive - Cogencis Information Services Ltd\Documents\Dec 24"
fund_path = r'C:\Users\rando\OneDrive\Documents\Dec 24'

In [50]:
def extract_data_relative_line(path: str, line_x: float, side: str):
    doc = fitz.open(path)
    pages = doc.page_count

    final_list = []

    for pgn in range(pages):
        page = doc[pgn]

        blocks = page.get_text("dict")["blocks"]
        extracted_blocks = []

        # Keep track of blocks to avoid duplicates
        added_blocks = set()

        for block in blocks:
            block_id = id(block)  # Unique identifier for the block

            for line in block.get("lines", []):
                for span in line.get("spans", []):
                    origin = span["origin"]
                    x0, _ = origin

                    # Check the side condition
                    if side == "left" and x0 < line_x and block_id not in added_blocks:
                        extracted_blocks.append(block)
                        added_blocks.add(block_id)  # Mark block as added
                    elif side == "right" and x0 > line_x and block_id not in added_blocks:
                        extracted_blocks.append(block)
                        added_blocks.add(block_id)  # Mark block as added

        # Sort extracted blocks
        sorted_blocks = sorted(extracted_blocks, key=lambda x: (x["bbox"][1], x["bbox"][0]))

        final_list.append({
            "pgn": pgn,
            "blocks": sorted_blocks
        })

    doc.close()

    return final_list


In [26]:
def extract_spans(data):
    spans = []
    for blocks in data:
        for block in blocks.get('blocks',[]):
            for line in block.get("lines", []):
                for span in line.get("spans", []):
                    spans.append({
                        "text": span.get("text"),
                        "color": span.get("color"),
                        "size": span.get("size")
                    })
    return spans

In [5]:
mutual_fund = Helper.get_fund_paths(fund_path)

In [11]:
sample_path  = mutual_fund['Groww Mutual Fund']

In [13]:
data = Helper.get_all_pdf_data(sample_path)
len(data)

53

In [51]:
extracted_data = extract_data_relative_line(sample_path,225,'left')

In [52]:
extracted_data[23]

{'pgn': 23,
 'blocks': [{'number': 1,
   'type': 0,
   'bbox': (33.015098571777344,
    36.55198669433594,
    294.7732849121094,
    62.516998291015625),
   'lines': [{'spans': [{'size': 19.0,
       'flags': 20,
       'font': 'NunitoSans7pt-ExtraBold',
       'color': -14475488,
       'ascender': 1.0110000371932983,
       'descender': -0.3529999852180481,
       'text': 'GROWW',
       'origin': (33.015098571777344, 55.80999755859375),
       'bbox': (33.015098571777344,
        36.60099792480469,
        121.91580200195312,
        62.516998291015625)},
      {'size': 19.0,
       'flags': 20,
       'font': 'NunitoSans7pt-ExtraBold',
       'color': -16723812,
       'ascender': 1.0110000371932983,
       'descender': -0.3529999852180481,
       'text': ' MULTICAP FUND',
       'origin': (121.91580200195312, 55.80999755859375),
       'bbox': (121.91580200195312,
        36.55198669433594,
        294.7732849121094,
        62.516998291015625)}],
     'wmode': 0,
     'dir': (1.

In [42]:
# Lines to be drawn (start and end points)
lines = [
    ((285, 0), (285, 1000)),# Vertical line
    #((420, 0), (420, 1000))
]

rectangles = [(0,5,225,812)]

sample_path  = mutual_fund['Groww Mutual Fund']
pages = [i for i in range(1,70,1)]
Helper.draw_lines_on_pdf(sample_path, lines, rectangles, pages, dry_path)
data = Helper.get_all_pdf_data(sample_path)

Modified PDF saved to: C:\Users\Kaustubh.keny\OneDrive - Cogencis Information Services Ltd\Documents\mywork-repo\output\DryRun.pdf
